In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata, select_images
from models import County, DataTrustedIdentifier

In [3]:
engine, session=db.connect()
engine.echo=False

2023-03-30 20:21:22,556 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-30 20:21:22,556 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 20:21:22,558 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-30 20:21:22,558 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 20:21:22,560 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-30 20:21:22,560 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
list_regions=['Norte', 'Nordeste', 'Centro-Oeste', 'Sul', 'Sudeste']
path_out = '/home/xandao/Imagens/region_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']

In [5]:
for color in list_color:
    for region in list_regions:
        for image_size in list_image_size:
            for minimum_image in list_minimum_image:
                query = session.query(County).filter(County.regiao.__eq__(region)).distinct().all()

                state_unaccented_lower, uf_unaccented_lower, county_unaccented_lower = db.get_state_uf_county(query)

                list_level = []
                list_path_images_final = []
                list_seq_final = []
                list_count_samples = []

                level=DataTrustedIdentifier.specific_epithet_trusted
                out=os.path.join(path_out, color.upper(), level.name, region, image_size, str(minimum_image))

                condition = sa.and_(DataTrustedIdentifier.country_trusted.__eq__('Brasil'),
                                    level.is_not(None),
                                    sa.or_(uf_unaccented_lower,
                                           state_unaccented_lower,
                                           county_unaccented_lower))

                query = db.get_records_group_by_level(condition, level, minimum_image, session)
                print('region: %s color: %s image_size: %s minimum_image: %d' % (region, color.upper(), image_size, minimum_image))
                select_images(color, image_size, list_count_samples, list_level, list_images_invalid, list_path_images_final, list_seq_final, minimum_image, session, query)

                list_f = copy_images(list_level, list_path_images_final, out)
                save_metadata(color, image_size, level, list_count_samples, list_f, list_level, list_path_images_final, list_seq_final, minimum_image, out, session, region=region)

/home/xandao/Documentos/piperaceae-sql/database.py:77: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query = session.query(*columns) \


region: Norte color: RGB image_size: 256 minimum_image: 5
total of levels: 110 total of images: 2682


,levels,count,dst,paths,seq
0,aduncum,58,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[8714, 9521, 11342, 11874, 11876, 11893, 12507..."
1,aequale,9,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[12577, 30534, 30546, 36793, 43763, 43769, 457..."
2,alata,8,/home/xandao/Imagens/region_dataset/RGB/specif...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[26378, 26385, 26457, 37031, 43122, 43125, 431..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,8714,Piper,aduncum,Piper,aduncum,[https://storage.googleapis.com/cria-zoomify/h...
1,1363,Peperomia,glabella,Peperomia,glabella,[https://storage.googleapis.com/cria-zoomify/h...
2,44254,Piper,coruscans,Piper,coruscans,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/region_dataset/RGB/specif...
color,rgb
image_size,256
minimum_image,5
level_name,specific_epithet_trusted
region,Norte
levels,110
total,2682
average,24.38


...

,0
dst,/home/xandao/Imagens/region_dataset/GRAYSCALE/...
color,grayscale
image_size,512
minimum_image,10
level_name,specific_epithet_trusted
region,Sudeste
levels,70
total,2622
average,37.46


region: Sudeste color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 42 total of images: 2263


,levels,count,dst,paths,seq
0,aduncum,94,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[7742, 7757, 8700, 8705, 8711, 9435, 9439, 957..."
1,alata,29,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 110..."
2,amalago,114,/home/xandao/Imagens/region_dataset/GRAYSCALE/...,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,"[2798, 7741, 7908, 7928, 8751, 8757, 8758, 876..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,19770,Ottonia,anisum,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,21142,Ottonia,propinqua,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


,0
dst,/home/xandao/Imagens/region_dataset/GRAYSCALE/...
color,grayscale
image_size,512
minimum_image,20
level_name,specific_epithet_trusted
region,Sudeste
levels,42
total,2263
average,53.88


In [6]:
session.close()
engine.dispose()